In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, KFold
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from scipy.stats import pearsonr
import torchbnn as bnn

In [2]:
#import creep data
creep_df = pd.read_csv('SS316_PI_dataset.csv')
creep_df

,Material,C,Si,Mn,P,S,Ni,Cr,Mo,Cu,...,B,N,Nb+Ta,Elongation,Reduction of Area,Stress,Temperature,Stacking Fault Energy,Predicted Creep Life,Log Creep Life
0,0.0,0.060,0.59,1.69,0.024,0.017,13.32,16.73,2.38,0.07,...,0.0010,0.0300,0.020,55.0,61,235,650,66.761225,1.438188,1.465383
1,1.0,0.050,0.52,1.51,0.021,0.010,13.21,16.42,2.34,0.14,...,0.0005,0.0340,0.010,68.0,78,157,700,66.602514,1.428500,1.537819
2,7.0,0.060,0.52,1.60,0.025,0.007,13.30,16.70,2.25,0.24,...,0.0008,0.0318,0.010,72.0,77,235,650,65.227932,1.471625,1.588832
3,18.0,0.012,0.56,0.81,0.024,0.004,10.67,16.39,2.11,0.27,...,0.0010,0.0810,0.001,106.0,80,177,700,58.700152,1.575937,1.591065
4,8.0,0.060,0.52,1.58,0.025,0.007,13.60,16.60,2.31,0.26,...,0.0007,0.0224,0.010,75.0,83,157,700,66.624772,1.566875,1.604226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,7.0,0.060,0.52,1.60,0.025,0.007,13.30,16.70,2.25,0.24,...,0.0008,0.0318,0.010,41.0,55,118,600,65.227932,5.178750,5.152857
613,6.0,0.070,0.61,1.65,0.025,0.007,13.60,16.60,2.33,0.26,...,0.0011,0.0250,0.010,26.0,39,108,600,66.647083,5.289000,5.184004
614,2.0,0.050,0.71,1.52,0.022,0.013,13.50,17.50,2.28,0.17,...,0.0013,0.0350,0.020,28.0,37,108,600,64.234884,5.247063,5.240086
615,6.0,0.070,0.61,1.65,0.025,0.007,13.60,16.60,2.33,0.26,...,0.0011,0.0250,0.010,25.0,23,41,700,66.647083,5.353813,5.250884


In [3]:
creep_df = creep_df.drop(columns=['Stacking Fault Energy', 'Predicted Creep Life'])
creep_df.head()

,Material,C,Si,Mn,P,S,Ni,Cr,Mo,Cu,Ti,Al,B,N,Nb+Ta,Elongation,Reduction of Area,Stress,Temperature,Log Creep Life
0,0.0,0.060,0.59,1.69,0.024,0.017,13.32,16.73,2.38,0.07,0.0110,0.015,0.0010,0.0300,0.020,55.0,61,235,650,1.465383
1,1.0,0.050,0.52,1.51,0.021,0.010,13.21,16.42,2.34,0.14,0.0110,0.018,0.0005,0.0340,0.010,68.0,78,157,700,1.537819
2,7.0,0.060,0.52,1.60,0.025,0.007,13.30,16.70,2.25,0.24,0.0600,0.020,0.0008,0.0318,0.010,72.0,77,235,650,1.588832
3,18.0,0.012,0.56,0.81,0.024,0.004,10.67,16.39,2.11,0.27,0.0006,0.011,0.0010,0.0810,0.001,106.0,80,177,700,1.591065
4,8.0,0.060,0.52,1.58,0.025,0.007,13.60,16.60,2.31,0.26,0.0290,0.021,0.0007,0.0224,0.010,75.0,83,157,700,1.604226


In [4]:
rm_state = 123
test_size = 0.6

X, X_test, y, y_test = train_test_split(np.array(creep_df.iloc[:, 0:19]), np.array(creep_df.iloc[:,19]), shuffle=True, test_size=test_size, random_state=rm_state)

idx = np.arange(len(y))

In [5]:
scaler = MinMaxScaler()

X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

In [6]:
train_ratio = 0.1

X_train, _, y_train, _, idx_train, idx_pool = train_test_split(X, y, idx, train_size=train_ratio, shuffle=True, random_state=rm_state)

In [7]:
X = X.astype(dtype=np.float32)
X_test =X_test.astype(np.float32)
X_train =X_train.astype(np.float32)

y = y.astype(dtype=np.float32)
y_test = y_test.astype(dtype=np.float32)
y_train = y_train.astype(dtype=np.float32)

X = torch.from_numpy(X)
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)
y_train = torch.from_numpy(y_train)
y = torch.from_numpy(y)

y_train = torch.unsqueeze(y_train, dim=1)
y_test = torch.unsqueeze(y_test, dim=1)
y = torch.unsqueeze(y, dim=1)

In [8]:
n_iter = 20
pcc_variance = []
r2_variance = []
rmse_variance = []
mae_variance = []
pcc_random = []
r2_random = []
num_training_data = []

X_train_var = X_train
X_train_ran = X_train
y_train_var = y_train
y_train_ran = y_train
idx_pool_var = idx_pool
idx_pool_ran = idx_pool
idx_train_var = idx_train
idx_train_ran = idx_train

#build the model 
model = nn.Sequential(
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.06, in_features=19, out_features=100),
        nn.ReLU(),
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.06, in_features=100, out_features=100),
        nn.ReLU(),
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.06, in_features=100, out_features=100),
        nn.ReLU(),
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.06, in_features=100, out_features=1),
)

mse_loss = nn.MSELoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 0.01
optimizer = optim.SGD(model.parameters(), lr=0.001, nesterov=True, momentum=0.95)

for i in range(n_iter):
    print(f"Performing iteration : {i}")

    if i != 0:
        # find 10 data points with the highest variance
        print()
   
        q_points_var = np.argpartition(y_pred_unc_pool_var, -10)[-10:]
        # indices of those points in idx_pool
        idx_pool_train_var = idx_pool_var[q_points_var]

        idx_train_var = np.append(idx_train_var, idx_pool_train_var)
        idx_pool_var = np.delete(idx_pool_var, q_points_var)
        X_train_var = X[idx_train_var]
        y_train_var = y[idx_train_var]

    print(f"Number of training data with variance: {len(idx_train_var)}")
    print(f"Number of pooling data with variance: {len(idx_pool_var)}")
    
    num_training_data.append(len(idx_train_var))


    for step in range(3000):


        y_pred_var = model(X_train_var)
        
        mse = mse_loss(y_pred_var, y_train_var)
        kl = kl_loss(model)
        cost = mse + kl_weight*kl

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

    if step % 1000==0:
        print('MSE : %2.2f, KL : %2.2f' % (mse.item(), kl.item()))

    n_samples = 1000

    # compute predictions
    y_preds_var = [model(X_test).clone().detach().numpy() for _ in range(n_samples)]
    y_preds_var = np.array(y_preds_var)
    # mean and standard deviation
    y_pred_test_var = np.mean(y_preds_var, axis=0)
    y_pred_unc_test_var = np.std(y_preds_var, axis=0)
    # y_test = y_test.detach().numpy()

    # get the idx pool var
    y_preds_pool_var = [model(X[idx_pool_var]).clone().detach().numpy() for _ in range(n_samples)]
    y_preds_pool_var = np.array(y_preds_pool_var)
    y_pred_unc_pool_var = np.squeeze(np.std(y_preds_pool_var, axis=0))

    print('PCC_test_score', pearsonr(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var))[0])
    print('R2_test_score', r2_score(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var)))
    print('RMSE', np.sqrt(mean_squared_error(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var))))
    print('MAE', np.mean(abs(np.squeeze(y_test.numpy()) - np.squeeze(y_pred_test_var))))

    pcc_variance.append(pearsonr(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var))[0])
    r2_variance.append(r2_score(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var)))
    rmse_variance.append(np.sqrt(mean_squared_error(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var))))
    mae_variance.append(np.mean(abs(np.squeeze(y_test.numpy())- np.squeeze(y_pred_test_var))))

Performing iteration : 0
Number of training data with variance: 24
Number of pooling data with variance: 222
PCC_test_score 0.6827626610596957
R2_test_score 0.14451402719339645
RMSE 0.83160305
MAE 0.6467633
Performing iteration : 1

Number of training data with variance: 34
Number of pooling data with variance: 212
PCC_test_score 0.7220342688830538
R2_test_score 0.3927859118683418
RMSE 0.70061684
MAE 0.5369056
Performing iteration : 2

Number of training data with variance: 44
Number of pooling data with variance: 202
PCC_test_score 0.827979971975104
R2_test_score 0.6701828678605897
RMSE 0.51635253
MAE 0.38921145
Performing iteration : 3

Number of training data with variance: 54
Number of pooling data with variance: 192
PCC_test_score 0.894923136021309
R2_test_score 0.7823026465455911
RMSE 0.41950414
MAE 0.3321034
Performing iteration : 4

Number of training data with variance: 64
Number of pooling data with variance: 182
PCC_test_score 0.9157783997641561
R2_test_score 0.8269587872971

'\n    # random sampling\n    if i != 0:\n        # select 10 random data points\n        q_points_ran = np.random.choice(np.arange(len(idx_pool_ran)), size=10)\n        # indices of those points in idx_pool\n        idx_pool_train_ran = idx_pool_ran[q_points_ran]\n\n        idx_train_ran = np.append(idx_train_ran, idx_pool_train_ran)\n        idx_pool_ran = np.delete(idx_pool_ran, q_points_ran)\n        X_train_ran = X[idx_train_ran]\n        y_train_ran = y[idx_train_ran]\n\n    print(f"Number of training data with random: {len(idx_train_ran)}")\n    print(f"Number of pooling data with random: {len(idx_pool_ran)}")\n\n    for step in range(3000):\n\n\n        y_pred_ran = model(X_train_ran)\n        mse = mse_loss(y_pred_ran, y_train_ran)\n        kl = kl_loss(model)\n        cost = mse + kl_weight*kl\n\n        optimizer.zero_grad()\n        cost.backward()\n        optimizer.step()\n\n    if step % 1000==0:\n        print(\'MSE : %2.2f, KL : %2.2f\' % (mse.item(), kl.item()))\n\n  

In [10]:
# plt.plot(num_iteration, r2_variance, 'b')
# plt.plot(num_iteration, r2_random, 'r')
# plt.show()
import pickle

In [16]:
with open('AL_BNN_VarInf.pkl', 'wb') as f:
    pickle.dump({'train_numbs':num_training_data, 'pcc':pcc_variance,'r2':r2_variance, 'rsme': rmse_variance, 'mae': mae_variance}, f)
    f.close()

In [19]:
pkl_file = open('AL_BNN_VarInf.pkl', 'rb')  
test = pickle.load(pkl_file)